#### 4.3 Create the county facts table
- Start with covid case data, split out date columns into a table with county-date-case rows
- Window with county partition and date sorting, calculate data by lagging one behind with 0 fill
- Do the same for covid death data, then join
- Same for all the weather data, keep joining
- Fill in 0 for all missing row fields
- Partition by county

##### Setup
I'm going to need Spark for this because I'll want to make use of some of its functionality, such as the ability to create temporary SQL views of my dataframes.

In [1]:
from setup import create_spark_session

spark = create_spark_session()

Imports and output paths:

In [2]:
import pandas as pd
from pyspark.sql import functions as F
from pyspark.sql.types import *
from pyspark.sql.window import Window

from clean import *
from etl import *

# For now, just locally, later on maybe write this to S3 instead
output_path = "output/"

Let's first load in the cleaned Covid data and inspect the schema:

In [3]:
covid_cases_df = load_covid_case_data(spark)

First, let's extract all the dates from the Covid-19 data set. This is so we can avoid having to cast the column names to dates constantly, instead we can do it once and cache them here.  
Apparently, I have to store them as unix timestamps; I tried keeping them as DateType, but when I later use them to construct the dataframe, it just gives me empty lists instead of datetimes.

In [4]:
unix_time = pd.Timestamp("1970-01-01")
second = pd.Timedelta('1s')

date_list = [(pd.to_datetime(c) - unix_time) // second for c in covid_cases_df.columns[5:]]
date_list[:5]

[1579651200, 1579737600, 1579824000, 1579910400, 1579996800]

We only need the FIPS codes and the time columns, let's extract these.

In [5]:
time_data_columns = covid_cases_df.columns[5:]
time_data_columns.insert(0, 'fips')
time_data_columns.insert(1, 'state')

time_data_columns[:5]

['fips', 'state', '1/22/20', '1/23/20', '1/24/20']

For each row in the dataframe, for each time column, we want to create one new row in a new dataframe where each row contains the FIPS code (to identify the county), the date of the column, and the case count for this county at that date.  
We store these as a tuple in the list so that we can insert each tuple as a row into a dataframe later on.

In [6]:
time_series = []

def extract_county_case_data(row):
    fips = time_data_columns[0]
    state = time_data_columns[1]
    for i in range(2, len(time_data_columns)):
        time_series.append((row[fips], row[state], date_list[i - 2], row[time_data_columns[i]]))

# I have no idea why the foreach doesn't work, I can only get it working if I collect the data
#covid_cases_df.limit(5).foreach(extract_county_case_data)

for row in covid_cases_df.collect():
    extract_county_case_data(row)

time_series[:5]

[(1001, 'Alabama', 1579651200, 0),
 (1001, 'Alabama', 1579737600, 0),
 (1001, 'Alabama', 1579824000, 0),
 (1001, 'Alabama', 1579910400, 0),
 (1001, 'Alabama', 1579996800, 0)]

We've got the values for the rows, now we just need to define the column names and build the dataframe.

In [7]:
time_series_columns = ['fips', 'state', 'timestamp', 'covid_case_total']

county_cases_df = spark.createDataFrame(time_series, time_series_columns)
county_cases_df.limit(5).show()

+----+-------+----------+----------------+
|fips|  state| timestamp|covid_case_total|
+----+-------+----------+----------------+
|1001|Alabama|1579651200|               0|
|1001|Alabama|1579737600|               0|
|1001|Alabama|1579824000|               0|
|1001|Alabama|1579910400|               0|
|1001|Alabama|1579996800|               0|
+----+-------+----------+----------------+



We could just keep the timestamps? I don't know if we need the datetimes since we'll fetch these from another table. I'll keep these in for now because it makes it easier for me to tell which dates I'm dealing with while debugging.

In [8]:
county_cases_df = county_cases_df.withColumn("date", F.from_unixtime("timestamp").cast(DateType()))
county_cases_df.limit(5).show()

+----+-------+----------+----------------+----------+
|fips|  state| timestamp|covid_case_total|      date|
+----+-------+----------+----------------+----------+
|1001|Alabama|1579651200|               0|2020-01-22|
|1001|Alabama|1579737600|               0|2020-01-23|
|1001|Alabama|1579824000|               0|2020-01-24|
|1001|Alabama|1579910400|               0|2020-01-25|
|1001|Alabama|1579996800|               0|2020-01-26|
+----+-------+----------+----------------+----------+



We only have the cumulative Covid-19 case counts, not the daily increase. Since we're most likely going to check the delta quite frequently, let's add that to the data set:

In [9]:
windowSpec = Window \
    .partitionBy(county_cases_df['fips']) \
    .orderBy(county_cases_df['timestamp'].asc())

''' This is more long-form, I wrote this because I thought that lagged numbers were bleeding across partition boundaries, but that doesn't seem to be the case.
county_cases_df = county_cases_df.withColumn('lag', F.lag(county_cases_df['covid_case_total']).over(windowSpec))

county_cases_df = county_cases_df.withColumn('covid_case_delta', \
    F.when(county_cases_df['lag'].isNull(), 0) \
    .otherwise(county_cases_df['covid_case_total'] - county_cases_df['lag']))

county_cases_df = county_cases_df.drop('lag')
'''

county_cases_df = county_cases_df.withColumn('covid_case_delta', \
    county_cases_df['covid_case_total'] - F.lag(county_cases_df['covid_case_total'], 1, 0).over(windowSpec))

county_cases_df.limit(5).show()

+----+--------+----------+----------------+----------+----------------+
|fips|   state| timestamp|covid_case_total|      date|covid_case_delta|
+----+--------+----------+----------------+----------+----------------+
|8075|Colorado|1579651200|               0|2020-01-22|               0|
|8075|Colorado|1579737600|               0|2020-01-23|               0|
|8075|Colorado|1579824000|               0|2020-01-24|               0|
|8075|Colorado|1579910400|               0|2020-01-25|               0|
|8075|Colorado|1579996800|               0|2020-01-26|               0|
+----+--------+----------+----------------+----------+----------------+



In [10]:
county_cases_df.where(county_cases_df['covid_case_delta'].isNull()).count()

0

So far, so good, no null entries in the delta column.

In [11]:
county_cases_df.agg({'covid_case_delta': 'max'}).show()

+---------------------+
|max(covid_case_delta)|
+---------------------+
|                29423|
+---------------------+



In [12]:
county_cases_df.agg({'covid_case_delta': 'min'}).show()

+---------------------+
|min(covid_case_delta)|
+---------------------+
|                -3059|
+---------------------+



Wait, what? Why do we have negative case counts here?

In [ ]:
county_cases_df.where(county_cases_df['covid_case_delta'] < 0).show()

In [ ]:
county_cases_df.where(county_cases_df['covid_case_delta'] < 0).count()

In [ ]:
county_cases_df.where(county_cases_df['covid_case_delta'] > 0).count()

There seem to be a fair few cases of this, but there aren't that many compared to the amount of actual increases, so it seems to work in most cases. It's also not just occurring once per FIPS code which might indicate an issue with my windowing logic.

At a guess, this is due to overreporting some cases one day and then correcting them the next day. These are case numbers as well, so it's possible that someone received a false positive and got scrubbed from the case counts again.

In [ ]:
county_cases_df.where(county_cases_df['covid_case_delta'] < -20).count()

In [ ]:
county_cases_df.where(county_cases_df['covid_case_delta'] < -20).show()

Ok, so there are still several hundred cases where more than 20 negative cases were reported. Still, this doesn't seem to be a bug, but just something inherent to the data. I checked a few of these by hand in the source data, and the data on days right after the decrease looks fine again, it jumps back up to the previous value and beyond. This makes me think that at least some of these are mistakes in data entry.

I'll attempt to fix up the broken deltas and totals by interpolating using the neighbouring rows.

In [13]:
adjusted_county_cases_df = county_cases_df.withColumn('lag', F.lag(county_cases_df['covid_case_total'], 1).over(windowSpec))
adjusted_county_cases_df = adjusted_county_cases_df.withColumn('lead', F.lead(county_cases_df['covid_case_total'], 1).over(windowSpec))
adjusted_county_cases_df = adjusted_county_cases_df.withColumn('next_delta', F.lead(county_cases_df['covid_case_delta'], 1).over(windowSpec))

# Get rid of overreporting
adjusted_county_cases_df = adjusted_county_cases_df.withColumn('covid_case_total', \
    F.when((adjusted_county_cases_df['next_delta'] >= 0) | adjusted_county_cases_df['lag'].isNull() | (adjusted_county_cases_df['lead'].isNull()), county_cases_df['covid_case_total']) \
    .otherwise(F.ceil((adjusted_county_cases_df['lead'] + adjusted_county_cases_df['lag']) / 2)))

adjusted_county_cases_df = adjusted_county_cases_df.withColumn('lag', F.lag(adjusted_county_cases_df['covid_case_total'], 1).over(windowSpec))

adjusted_county_cases_df = adjusted_county_cases_df.withColumn('covid_case_delta', \
    adjusted_county_cases_df['covid_case_total'] - adjusted_county_cases_df['lag'])

adjusted_county_cases_df = adjusted_county_cases_df.drop('lag').drop('lead').drop('next_delta')

In [ ]:
adjusted_county_cases_df.agg({'covid_case_delta': 'min'}).show()

In [ ]:
adjusted_county_cases_df.where(adjusted_county_cases_df['covid_case_delta'] < 0).count()

In [ ]:
adjusted_county_cases_df.where(adjusted_county_cases_df['covid_case_delta'] < 0).show()

In [ ]:
adjusted_county_cases_df.where(adjusted_county_cases_df['covid_case_delta'] < -20).count()

In [ ]:
adjusted_county_cases_df.where(adjusted_county_cases_df['covid_case_delta'] < -20).show()

This seems to have worked somewhat. The maximum drop is reduced, and we have less than one-fifth the amount of negative deltas as before, and only 12 outliers.  
Spot checks on the source data shows that we eliminated the one-off drops, but there are still drops when the incorrect data persists over multiple days. I don't feel comfortable messing with the data this much since multi-day data entry issues seem unlikelier than that the case count did in fact change, due to correction, false positives, etc.

Now let's do it all again for the death data:

In [14]:
covid_deaths_df = load_covid_deaths_data(spark)
covid_deaths_df.limit(1).show()

+----+-----------+-------+-----------+------------+----------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+------+------+------+------+------+------+------+------+------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+------+------+------+------+------+------+------+------+------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+------+------+------+------+------+------+------+------+------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+------+------+------+------+------+------+------+------+------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+------

In [15]:
date_list = [(pd.to_datetime(c) - unix_time) // second for c in covid_deaths_df.columns[6:]]

time_data_columns = covid_deaths_df.columns[6:]
time_data_columns.insert(0, 'fips')

time_series = []

def extract_county_death_data(row):
    fips = time_data_columns[0]
    for i in range(1, len(time_data_columns)):
        time_series.append((row[fips], date_list[i - 1], row[time_data_columns[i]]))

for row in covid_deaths_df.collect():
    extract_county_death_data(row)

time_series_columns = ["fips", "timestamp", "covid_death_total"]

county_deaths_df = spark.createDataFrame(time_series, time_series_columns)

windowSpec = Window \
    .partitionBy(county_deaths_df['fips']) \
    .orderBy(county_deaths_df['timestamp'].asc())

county_deaths_df = county_deaths_df.withColumn('lag', F.lag(county_deaths_df['covid_death_total'], 1).over(windowSpec))
county_deaths_df = county_deaths_df.withColumn('lead', F.lead(county_deaths_df['covid_death_total'], 1).over(windowSpec))

# Populate deltas
county_deaths_df = county_deaths_df.withColumn('covid_death_delta', \
    F.when(county_deaths_df['lag'].isNull(), 0) \
    .otherwise(county_deaths_df['covid_death_total'] - county_deaths_df['lag']))

county_deaths_df = county_deaths_df.withColumn('next_delta', F.lead(county_deaths_df['covid_death_delta'], 1).over(windowSpec))

# Fix overreporting
county_deaths_df = county_deaths_df.withColumn('covid_death_total', \
    F.when((county_deaths_df['next_delta'] >= 0) | (county_deaths_df['lag'].isNull() | (county_deaths_df['lead'].isNull())), county_deaths_df['covid_death_total']) \
    .otherwise(F.ceil((county_deaths_df['lead'] + county_deaths_df['lag']) / 2)))

# Recalculate deltas
county_deaths_df = county_deaths_df.withColumn('lag', F.lag(county_deaths_df['covid_death_total'], 1).over(windowSpec))
county_deaths_df = county_deaths_df.withColumn('covid_death_delta', \
    F.when(county_deaths_df['lag'].isNull(), 0) \
    .otherwise(county_deaths_df['covid_death_total'] - county_deaths_df['lag']))

county_deaths_df = county_deaths_df.drop('lag').drop('lead').drop('next_delta')

In [16]:
county_deaths_df.agg({'covid_death_total': 'min'}).show()

+----------------------+
|min(covid_death_total)|
+----------------------+
|                     0|
+----------------------+



In [ ]:
county_deaths_df.agg({'covid_death_total': 'max'}).show()

In [ ]:
county_deaths_df.where(county_deaths_df['covid_death_delta'].isNull()).count()

In [17]:
county_deaths_df.agg({'covid_death_delta': 'min'}).show()

+----------------------+
|min(covid_death_delta)|
+----------------------+
|                   -47|
+----------------------+



In [ ]:
county_deaths_df.agg({'covid_death_delta': 'max'}).show()

In [ ]:
county_deaths_df.where(county_deaths_df['covid_death_delta'] < 0).count()

In [ ]:
county_deaths_df.where(county_deaths_df['covid_death_delta'] < -10).count()

In [ ]:
county_deaths_df.where(county_deaths_df['covid_death_delta'] < -10).show()

Death data looks good, we do see some negative deltas again but the vast majority is very small, most likely due to corrections, overreporting, etc.

In [18]:
county_facts_df = county_cases_df.join(county_deaths_df, on=["fips", "timestamp"], how="left")
county_facts_df.limit(5).show()

+----+----------+-------+----------------+----------+----------------+-----------------+-----------------+
|fips| timestamp|  state|covid_case_total|      date|covid_case_delta|covid_death_total|covid_death_delta|
+----+----------+-------+----------------+----------+----------------+-----------------+-----------------+
|1001|1586044800|Alabama|              12|2020-04-05|               0|                0|                0|
|1003|1595980800|Alabama|            3001|2020-07-29|             128|               20|                3|
|1003|1599609600|Alabama|            4796|2020-09-09|              41|               42|                0|
|1005|1603238400|Alabama|            1007|2020-10-21|              14|                9|                0|
|1005|1605744000|Alabama|            1145|2020-11-19|               8|               10|                0|
+----+----------+-------+----------------+----------+----------------+-----------------+-----------------+



In [ ]:
county_facts_df.where(county_facts_df['fips'] == 1001).count()

Great, this seems to have worked! So now we have the Covid-19 case and death counts.

Next up, do something similar for the weather data.

In [19]:
tMin_df, tMax_df, cloud_df, wind_df = load_weather_data(spark)
tMin_df.limit(1).show()

+----+-----------+-------+-----------+------------+------+------+------+------+------+------+------+------+------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+------+------+------+------+------+------+------+------+------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+------+------+------+------+------+------+------+------+------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+------+------+------+------+------+------+------+------+------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+------+------+------+----

In [20]:
def transform_weather_data(df, column_name):
    date_list = [(pd.to_datetime(c) - unix_time) // second for c in df.columns[5:]]

    time_data_columns = df.columns[5:]
    time_data_columns.insert(0, 'fips')

    time_series = []

    def extract_weather_data(row):
        fips = time_data_columns[0]
        for i in range(1, len(time_data_columns)):
            time_series.append((row[fips], date_list[i - 1], float(row[time_data_columns[i]])))

    for row in df.collect():
        extract_weather_data(row)

    time_series_columns = ["fips", "timestamp", column_name]

    df = spark.createDataFrame(time_series, time_series_columns)
    
    return df

transformed_tMin_df = transform_weather_data(tMin_df, "min_temp")
transformed_tMax_df = transform_weather_data(tMax_df, "max_temp")
transformed_cloud_df = transform_weather_data(cloud_df, "cloud_cover")
transformed_wind_df = transform_weather_data(wind_df, "wind")

In [21]:
transformed_tMin_df.limit(5).show()

+----+----------+--------+
|fips| timestamp|min_temp|
+----+----------+--------+
|1001|1577836800|     0.0|
|1001|1577923200|     8.0|
|1001|1578009600|    16.0|
|1001|1578096000|    11.0|
|1001|1578182400|     0.0|
+----+----------+--------+



In [22]:
county_facts_df = county_facts_df.join(transformed_tMin_df, on=["fips", "timestamp"], how="left")
county_facts_df = county_facts_df.join(transformed_tMax_df, on=["fips", "timestamp"], how="left")
county_facts_df = county_facts_df.join(transformed_cloud_df, on=["fips", "timestamp"], how="left")
county_facts_df = county_facts_df.join(transformed_wind_df, on=["fips", "timestamp"], how="left")
county_facts_df.limit(5).show()

+----+----------+-------+----------------+----------+----------------+-----------------+-----------------+--------+--------+-----------+----+
|fips| timestamp|  state|covid_case_total|      date|covid_case_delta|covid_death_total|covid_death_delta|min_temp|max_temp|cloud_cover|wind|
+----+----------+-------+----------------+----------+----------------+-----------------+-----------------+--------+--------+-----------+----+
|1001|1586044800|Alabama|              12|2020-04-05|               0|                0|                0|    15.0|    27.0|       59.0| 1.0|
|1003|1595980800|Alabama|            3001|2020-07-29|             128|               20|                3|    23.0|    30.0|       70.0| 2.0|
|1003|1599609600|Alabama|            4796|2020-09-09|              41|               42|                0|   20.85|   32.28|       14.0|1.88|
|1005|1603238400|Alabama|            1007|2020-10-21|              14|                9|                0|   17.54|   28.84|       12.0|2.65|
|1005|

In [23]:
county_facts_df = county_facts_df.drop('date')
county_facts_df.limit(5).show()

+----+----------+-------+----------------+----------------+-----------------+-----------------+--------+--------+-----------+----+
|fips| timestamp|  state|covid_case_total|covid_case_delta|covid_death_total|covid_death_delta|min_temp|max_temp|cloud_cover|wind|
+----+----------+-------+----------------+----------------+-----------------+-----------------+--------+--------+-----------+----+
|1001|1586044800|Alabama|              12|               0|                0|                0|    15.0|    27.0|       59.0| 1.0|
|1003|1595980800|Alabama|            3001|             128|               20|                3|    23.0|    30.0|       70.0| 2.0|
|1003|1599609600|Alabama|            4796|              41|               42|                0|   20.85|   32.28|       14.0|1.88|
|1005|1603238400|Alabama|            1007|              14|                9|                0|   17.54|   28.84|       12.0|2.65|
|1005|1605744000|Alabama|            1145|               8|               10|      

In [ ]:
county_facts_df.write.partitionBy('fips').mode('overwrite').parquet(output_path + "county_facts.parquet")